In [1]:
from pynq.overlays.base import BaseOverlay
import time
base = BaseOverlay("base.bit")
import threading
import pynq.lib.rgbled as rgbled
buttons = base.btns_gpio

In [2]:
#define the locks and LEDs
philosophers = [base.leds[0] , base.leds[1] , base.leds[2] , base.leds[3] , rgbled.RGBLED(4)]
forks = [threading.Lock() , threading.Lock() , threading.Lock() , threading.Lock() , threading.Lock()]
is_napping = [False, False, False, False, False]
#is_napping = [True, True, True, True, True]

condition = True
for i in range(4):
    philosophers[i].off()
philosophers[4].write(0x0)
rgbled.RGBLED(5).write(0x0)

def rgbtoggle(num_blinksrgb, duration_rgb, k): #will need the index i and the array philosophers
    global philosophers, condition
    for x in range(num_blinksrgb):
        if philosophers[k].read() != 0:
            philosophers[k].write(0x0)
        else: #if off, then write green/yellow
            philosophers[k].write(0x6)
        time.sleep(duration_rgb)
        if condition == False:
            break

def blink(num_blinks, duration, n): #will need the # of blinks, duration of blinks, as well as index of LEDs (i) as n
    global philosophers, condition
    if n != 4: #check if not rgb led before for loop so only checks once per blink() call.
        for j in range(num_blinks):
            philosophers[n].toggle()
            time.sleep(duration)
            if condition == False:
                break
        philosophers[n].off()
    else:
        rgbtoggle(num_blinks, duration, n)
        philosophers[n].write(0x0)
        
def terminate():
    global condition
    while True:
        if buttons.read() != 0:
            if condition == False:
                condition = True
            else:
                condition = False
                #write all LEDs off too?
        time.sleep(0) #yield

In [ ]:
#sleep is 1.5s, eat is 1s, check for fork every 1s
def philosopher_state(_left_fork, _right_fork, index): #will need the philosophers arrays, as well as index i
    global philosophers, is_napping
    while condition:
        fork1 = _left_fork.acquire(False)
        fork2 = _right_fork.acquire(False)
        if is_napping[index] == True:
            _left_fork.release()
            _right_fork.release()
            is_napping[index] = False
            print("Philosopher {} is now napping".format(index))
            blink(6, .25, index) #sleep is 1.5s
            time.sleep(0) # yield
        else: #Either eating or starving
            if(fork1 and fork2): 
                print("Philosopher {} has both forks and is eating".format(index))
                blink(30, .10, index)
                #release the forks
                _left_fork.release()
                _right_fork.release()
                is_napping[index] = True
                #print("Philosopher {} has released both forks".format(index))
                time.sleep(0)
            else:
                print("Philosopher {} is starving".format(index))
                if (fork1 == True):
                    _left_fork.release()
                elif (fork2 == True):
                    _right_fork.release()
                time.sleep(.5)
    
threads = [] #define the thread numbers
for i in range(5):
    t = threading.Thread(target=philosopher_state, args=(forks[(i-1)%5], forks[i], i))
    threads.append(t) #creates the new thread number
    t.start()

t_terminate = threading.Thread(target=terminate, args=())
threads.append(t_terminate)
t_terminate.start()

for t in threads:
    name = t.getName()
    t.join()
    print('{} joined'.format(name))

Philosopher 0 has both forks and is eating
Philosopher 1 is starving
Philosopher 2 has both forks and is eating
Philosopher 3 is starving
Philosopher 4 is starving


/tmp/ipykernel_4084/1052676621.py:43: DeprecationWarning: getName() is deprecated, get the name attribute instead
  name = t.getName()


Philosopher 1 is starving
Philosopher 3 is starving
Philosopher 4 is starving
Philosopher 4 is starving
Philosopher 3 is starving
Philosopher 1 is starving
Philosopher 4 is starvingPhilosopher 3 is starving
Philosopher 1 is starving

Philosopher 3 is starving
Philosopher 1 is starving
Philosopher 4 is starving
Philosopher 1 is starvingPhilosopher 3 is starving
Philosopher 4 is starving

Philosopher 3 is starvingPhilosopher 4 is starving

Philosopher 1 is starving
Philosopher 4 is starvingPhilosopher 3 is starving
Philosopher 2 is now napping

Philosopher 1 is starving
Philosopher 0 is now napping
Philosopher 3 has both forks and is eating
Philosopher 4 is starving
Philosopher 1 has both forks and is eating
Philosopher 4 is starving
Philosopher 4 is starving
Philosopher 2 is starving
Philosopher 0 is starvingPhilosopher 4 is starving

Philosopher 2 is starving
Philosopher 4 is starving
Philosopher 0 is starving
Philosopher 2 is starving
Philosopher 4 is starving
Philosopher 0 is starvin